In [110]:
import sklearn
import nltk
import pandas as pd
import numpy as np
import tensorflow_datasets as tfds
from wordcloud import WordCloud, STOPWORDS
from nltk import word_tokenize
from nltk.util import ngrams
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt 



In [65]:
train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], #imports the IMBD dataset from tensorflow, code from https://www.tensorflow.org/hub/tutorials/tf2_text_classification
                                  batch_size=-1, as_supervised=True)

train_examples, train_labels = tfds.as_numpy(train_data)
test_examples, test_labels = tfds.as_numpy(test_data)



In [141]:
print("Training entries: {}, test entries: {}".format(len(train_examples), len(test_examples)))
full_dataset = np.concatenate((train_examples, test_examples), axis=0) #concatenate so that we can use full dataset for unigrams etc

Training entries: 25000, test entries: 25000


In [142]:
df = pd.DataFrame(data=full_dataset, index=None, columns=None) #creates a pandas dataframe of every sentence in the dataset, then turns the dataframe from a numpy array type into a string.  

df_str = df[0].str.decode("utf-8")
df_str.name = 'sentence' #names the dataframe
df_str

0        This was an absolutely terrible movie. Don't b...
1        I have been known to fall asleep during films,...
2        Mann photographs the Alberta Rocky Mountains i...
3        This is the kind of film for a snowy Sunday af...
4        As others have mentioned, all the women that g...
                               ...                        
49995    Feeling Minnesota is not really a road movie, ...
49996    This is, without doubt, one of my favourite ho...
49997    Most predicable movie I've ever seen...extreme...
49998    It's exactly what I expected from it. Relaxing...
49999    They just don't make cartoons like they used t...
Name: sentence, Length: 50000, dtype: object

In [ ]:
nltk.download('punkt') #for use in finding ngrams, code from practical
from nltk import word_tokenize
from nltk import ngrams

In [101]:
def extract_unigram(sentences): #extracts unigrams, code from practical

  tokens = []
  for sentence in df_str:
    tok = word_tokenize(sentence)
    for t in tok:
      tokens.append(t)
  return tokens

In [129]:
def extract_bigram(sentences): #extracts bigrams, code from practical

  all_bigrams = []
  for sentence in df_str:
    token = word_tokenize(sentence)
    bigrams = ngrams(token,2)
    for b in bigrams:
      all_bigrams.append(b)
  return all_bigrams

In [ ]:
extracted_bigrams = extract_bigram(df_str)
extracted_bigrams

In [ ]:
extracted_unigrams = extract_unigram(df_str)
extracted_unigrams


In [134]:
str_unigrams=(" ").join(extracted_unigrams) #generates a wordcloud to show most used unigrams
stopwords_set = {"br", ",", "<", ">", ".", ":"} #"words" to be excluded from wordcloud
#stopwords = set(STOPWORDS) commented out as it excluded common words we could want
wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                stopwords = stopwords_set, 
                min_font_size = 10,).generate(str_unigrams) 


In [ ]:
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
  
plt.show() 